In [ ]:
import pandas as pd
import numpy as np
import config as cf
import matplotlib.pyplot as plt

### Loading data

In [ ]:
%store -r tickers

In [ ]:
tickers

In [ ]:
dfs = dict()
print("...loading...")
for ticker in tickers:
    symbol = ticker["symbol"]
    if str(symbol).endswith("USDT"):
        print(symbol, end=" ")
        filename = f"{cf.AGGREGATED_DATA_PATH}futures/{symbol}-1d.h5"
        df = pd.read_hdf(filename)
        dfs[symbol] = df

In [ ]:
len(dfs)

##### Pick AAAUSDT symbol which has history data length greater than 600 days

In [ ]:
len_list = []
for symbol in dfs.keys():
    len_list.append(len(dfs[symbol]))
df = pd.DataFrame({"symbol": dfs.keys(), "length": len_list})

In [ ]:
len(df[df["length"]>600])

##### The Strategy is we arbitrage 5 pairs, with long top 5 that has the highest accumulated return in the last period, and short 5 pairs that has the lowest cummulated return in the last period.

##### Prepare data

In [ ]:
def run_strategy(
        trade_duration:int,
        lookback_period:int = 7,
        holding_period:int = 3,
        avoid_top:int = 3,
        n_pairs:int = 5,
        balance:int =100_000,
        trading_fee:float=0.0004):
    
    selected_symbols = df[df["length"]>trade_duration+lookback_period+holding_period + 50]["symbol"].values.tolist()
    for symbol in selected_symbols:
        dfs[symbol][f"log_return"] = np.log(dfs[symbol]["Close"].pct_change()+1)
        dfs[symbol][f"{symbol}"] = dfs[symbol]["log_return"].rolling(lookback_period).sum()
        dfs[symbol][f"{symbol}_future_return"] = dfs[symbol]["log_return"].shift(-holding_period).rolling(holding_period).sum()
        
    lookback_df = pd.concat([dfs[symbol][f"{symbol}"] for symbol in selected_symbols],axis=1)
    lookahead_df = pd.concat([dfs[symbol][f"{symbol}_future_return"] for symbol in selected_symbols],axis=1)

    lookback_df.sort_index(ascending=True,inplace=True)
    lookback_df.fillna(method="ffill", inplace = True)
    lookback_df.dropna(inplace=True)

    lookahead_df.sort_index(ascending=True,inplace=True)
    lookahead_df.fillna(method="ffill", inplace = True)
    lookahead_df.dropna(inplace=True)
    
    trade_start = max(lookback_df.index.min(),lookahead_df.index.min())
    trade_end = min(lookback_df.index.max(),lookahead_df.index.max())

    
    #Check if any missing open-time index
    for i in range (trade_start,trade_end,24*60*60*1000):
        if not i in lookback_df.index:
            print(f"miss {i}")
            
    for i in range (trade_start,trade_end,24*60*60*1000):
        if not i in lookahead_df.index:
            print(f"miss {i}")

    
    trade_df = pd.DataFrame({"open":[0],"long":[0],"short":[0],\
        "long_profit":[0],"short_profit":[0],"cost":[0],"balance":[balance]})

    balance = balance
    profit_before_cost = 0

    for open_time in range(trade_start,trade_end+1, holding_period * 24 * 60 * 60 * 1000):
        sum_long_profit = 0
        sum_short_profit = 0
        day_df = lookback_df.loc[open_time,:].to_frame(name="value").sort_values(by="value")
        nlargest = day_df.iloc[-n_pairs-avoid_top:-avoid_top-1].index.to_list()
        nlowest = day_df.iloc[avoid_top:n_pairs+avoid_top+1].index.to_list()   
        long_profits = []
        short_profits = []
        long_symbols = []
        short_symbols = []
        trading_costs = []
        balances = []

        invest_amount = balance/(2*n_pairs)
        
        sum_long_profit = 0
        sum_short_profit = 0
        sum_trading_cost = 0
        
        for symbol in nlargest:
            cumsum_log_return = lookahead_df.loc[open_time,:][f"{symbol}_future_return"]
            profit_before_cost = invest_amount * (np.exp(cumsum_log_return)-1)
            cost = invest_amount * 2 * trading_fee
            profit = profit_before_cost - cost
            long_profits.append(profit)
            short_profits.append(0)
            long_symbols.append(symbol)
            short_symbols.append("")
            sum_long_profit+=profit
            sum_trading_cost += cost
            trading_costs.append(cost)
            balance += profit
            balances.append(balance)
        
        for symbol in nlowest:
            cumsum_log_return = lookahead_df.loc[open_time,:][f"{symbol}_future_return"]
            profit_before_cost = invest_amount * -(np.exp(cumsum_log_return)-1)
            cost = invest_amount * 2 * trading_fee
            profit = profit_before_cost - cost
            short_profits.append(profit)
            long_profits.append(0)
            short_symbols.append(symbol)
            long_symbols.append("")
            sum_short_profit+= profit
            sum_trading_cost+= cost
            trading_costs.append(cost)
            balance += profit
            balances.append(balance)

        times = [open_time for i in range(len(nlargest)+len(nlowest))]

        row =pd.DataFrame({"open": times, "long":long_symbols,\
            "short":short_symbols,"long_profit":long_profits,"short_profit":short_profits, \
                "cost":trading_costs,"balance":balances})
        
        trade_df = pd.concat([trade_df,row],ignore_index=True)

    trade_df["cumsum_long_profit"] = trade_df["long_profit"].cumsum()
    trade_df["cumsum_short_profit"] = trade_df["short_profit"].cumsum()
    trade_df["cumsum_trading_cost"] = trade_df["cost"].cumsum()
    
    return trade_df

def plot_results(trade_df):
    plt.figure(figsize=(16,9),dpi=720)
    plt.plot(trade_df.index,trade_df["balance"],c="b",linewidth=1,label="balance")
    plt.plot(trade_df.index,trade_df["cumsum_long_profit"],c="g",linewidth=1,label="cumsum_long_profit")
    plt.plot(trade_df.index,trade_df["cumsum_short_profit"],c="r",linewidth=1,label="cumsum_short_profit")
    plt.plot(trade_df.index,trade_df["cumsum_trading_cost"],c="y",linewidth=1,label="cumsum_trading_cost")
    plt.legend()
    plt.show()

In [24]:
trade_df = run_strategy(
    trade_duration=90,
    lookback_period=14,
    holding_period=3,
    avoid_top=0,
    n_pairs=5,
    balance=1_000_000,
    trading_fee=0.0004
)
plot_results(trade_df)